# Connect Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/badminton/annotation_pipeline/shuttle

/content/gdrive/MyDrive/badminton/shuttle


# Install packages
Instructions from https://nol.cs.nctu.edu.tw:234/open-source/TrackNetv2/tree/master

In [ ]:
!pip install pyqt5
!pip install pandas
!pip install PyMySQL
!pip install opencv-python
!pip install imutils
!pip install Pillow
!pip install piexif
!pip install -U scikit-learn
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.4 MB 26.1 MB/s 
     |████████████████████████████████| 344 kB 60.7 MB/s 
     |████████████████████████████████| 59.9 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 2.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-

In [ ]:
!git clone https://nol.cs.nctu.edu.tw:234/open-source/TrackNetv2
%cd TrackNetv2/3_in_3_out/

/content/gdrive/MyDrive/badminton/shuttle/TrackNetv2/3_in_3_out


## Function for shuttle prediction tracking

In [ ]:
import sys
import getopt
import numpy as np
import os
from glob import glob
import piexif
import tensorflow
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import *
from keras.layers import *
from TrackNet3 import TrackNet3
import keras.backend as K
from keras import optimizers
import tensorflow as tf
import cv2
from os.path import isfile, join
from PIL import Image
import time
BATCH_SIZE=1
HEIGHT=288
WIDTH=512
#HEIGHT=360
#WIDTH=640
sigma=2.5
mag=1

def genHeatMap(w, h, cx, cy, r, mag):
	if cx < 0 or cy < 0:
		return np.zeros((h, w))
	x, y = np.meshgrid(np.linspace(1, w, w), np.linspace(1, h, h))
	heatmap = ((y - (cy + 1))**2) + ((x - (cx + 1))**2)
	heatmap[heatmap <= r**2] = 1
	heatmap[heatmap > r**2] = 0
	return heatmap*mag

#time: in milliseconds
def custom_time(time):
	remain = int(time / 1000)
	ms = (time / 1000) - remain
	s = remain % 60
	s += ms
	remain = int(remain / 60)
	m = remain % 60
	remain = int(remain / 60)
	h = remain
	#Generate custom time string
	cts = ''
	if len(str(h)) >= 2:
		cts += str(h)
	else:
		for i in range(2 - len(str(h))):
			cts += '0'
		cts += str(h)
	
	cts += ':'

	if len(str(m)) >= 2:
		cts += str(m)
	else:
		for i in range(2 - len(str(m))):
			cts += '0'
		cts += str(m)

	cts += ':'

	if len(str(int(s))) == 1:
		cts += '0'
	cts += str(s)

	return cts


#Loss function
def custom_loss(y_true, y_pred):
	loss = (-1)*(K.square(1 - y_pred) * y_true * K.log(K.clip(y_pred, K.epsilon(), 1)) + K.square(y_pred) * (1 - y_true) * K.log(K.clip(1 - y_pred, K.epsilon(), 1)))
	return K.mean(loss)


def predict_shuttle(videoName, load_weights, out_vidname, out_csvname):
  model = load_model(load_weights, custom_objects={'custom_loss':custom_loss})
  #model.summary()
  print('Beginning predicting......')

  start = time.time()

  f = open(out_csvname, 'w')
  f.write('Frame,Visibility,X,Y,Time\n')

  cap = cv2.VideoCapture(videoName)

  success, image1 = cap.read()
  frame_time1 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))
  success, image2 = cap.read()
  frame_time2 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))
  success, image3 = cap.read()
  frame_time3 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))

  ratio = image1.shape[0] / HEIGHT

  size = (int(WIDTH*ratio), int(HEIGHT*ratio))
  fps = 30

  if videoName[-3:] == 'avi':
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
  elif videoName[-3:] == 'mp4':
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  else:
    print('usage: video type can only be .avi or .mp4')
    exit(1)

  out = cv2.VideoWriter(out_vidname, fourcc, fps, size)

  count = 0

  while success:
    unit = []
    #Adjust BGR format (cv2) to RGB format (PIL)
    x1 = image1[...,::-1]
    x2 = image2[...,::-1]
    x3 = image3[...,::-1]
    #Convert np arrays to PIL images
    x1 = array_to_img(x1)
    x2 = array_to_img(x2)
    x3 = array_to_img(x3)
    #Resize the images
    x1 = x1.resize(size = (WIDTH, HEIGHT))
    x2 = x2.resize(size = (WIDTH, HEIGHT))
    x3 = x3.resize(size = (WIDTH, HEIGHT))
    #Convert images to np arrays and adjust to channels first
    x1 = np.moveaxis(img_to_array(x1), -1, 0)
    x2 = np.moveaxis(img_to_array(x2), -1, 0)
    x3 = np.moveaxis(img_to_array(x3), -1, 0)
    #Create data
    unit.append(x1[0])
    unit.append(x1[1])
    unit.append(x1[2])
    unit.append(x2[0])
    unit.append(x2[1])
    unit.append(x2[2])
    unit.append(x3[0])
    unit.append(x3[1])
    unit.append(x3[2])
    unit=np.asarray(unit)	
    unit = unit.reshape((1, 9, HEIGHT, WIDTH))
    unit = unit.astype('float32')
    unit /= 255
    y_pred = model.predict(unit, batch_size=BATCH_SIZE)
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype('float32')
    h_pred = y_pred[0]*255
    h_pred = h_pred.astype('uint8')
    for i in range(3):
      if i == 0:
        frame_time = frame_time1
        image = image1
      elif i == 1:
        frame_time = frame_time2
        image = image2
      elif i == 2:
        frame_time = frame_time3	
        image = image3

      if np.amax(h_pred[i]) <= 0:
        f.write(str(count)+',0,0,0,'+frame_time+'\n')
        out.write(image)
      else:	
        #h_pred
        (cnts, _) = cv2.findContours(h_pred[i].copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        rects = [cv2.boundingRect(ctr) for ctr in cnts]
        max_area_idx = 0
        max_area = rects[max_area_idx][2] * rects[max_area_idx][3]
        for i in range(len(rects)):
          area = rects[i][2] * rects[i][3]
          if area > max_area:
            max_area_idx = i
            max_area = area
        target = rects[max_area_idx]
        (cx_pred, cy_pred) = (int(ratio*(target[0] + target[2] / 2)), int(ratio*(target[1] + target[3] / 2)))

        f.write(str(count)+',1,'+str(cx_pred)+','+str(cy_pred)+','+frame_time+'\n')
        image_cp = np.copy(image)
        cv2.circle(image_cp, (cx_pred, cy_pred), 5, (0,0,255), -1)
        out.write(image_cp)
      count += 1
    success, image1 = cap.read()
    frame_time1 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))
    success, image2 = cap.read()
    frame_time2 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))
    success, image3 = cap.read()
    frame_time3 = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))

  f.close()
  out.release()
  end = time.time()
  print('Prediction time:', end-start, 'secs')
  print('Done......')

# Demo track shuttle

In [ ]:
%cd TrackNetv2/3_in_3_out/

[Errno 2] No such file or directory: 'TrackNetv2/3_in_3_out/'
/content/gdrive/MyDrive/badminton/shuttle/TrackNetv2/3_in_3_out


In [ ]:
!python predict3.py --video_name='/content/gdrive/MyDrive/badminton/shuttle/prodoub3.mp4' --load_weights='model906_30'

Traceback (most recent call last):
  File "predict3.py", line 7, in <module>
    from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
ImportError: cannot import name 'array_to_img' from 'keras.preprocessing.image' (/usr/local/lib/python3.7/dist-packages/keras/preprocessing/image.py)


In [ ]:
vid = '/content/gdrive/MyDrive/badminton/shuttle/prodoub3.mp4'
vidout = '/content/gdrive/MyDrive/badminton/shuttle/prodoub3_shuttle.mp4'
csvout = '/content/gdrive/MyDrive/badminton/shuttle/prodoub3_shuttle.csv'
predict_shuttle(vid, 'model906_30', vidout, csvout)

Beginning predicting......
1/1 [==============================] - 0s 24ms/step
Prediction time: 131.27257466316223 secs
Done......


## Run shuttle tracking and save detections in csv, plus output video

In [ ]:
import os 

# get vid paths
vidpaths = []
dataset_rootdir = '/content/gdrive/MyDrive/badminton/difficult_dataset/' #'../profession_dataset/'
match_dirs = [dataset_rootdir + matchdir for matchdir in sorted(os.listdir(dataset_rootdir))]

for matchdir in match_dirs:
  viddir = matchdir + '/rally_video/'
  vid_dirs = [viddir + vid for vid in os.listdir(viddir) if vid.endswith(".mp4")]
  vidpaths.append(sorted(vid_dirs))

for i in range(len(vidpaths)):
  print(vidpaths[i][0])
  for j in range(len(vidpaths[i])):
    vid = vidpaths[i][j]
    out_dir = 'shuttle_difficult/' + vid.split('/')[-3] + '/'
    if not os.path.exists(out_dir):
      os.makedirs(out_dir)
      
    basename = vid.split('/')[-1]

    vidout = out_dir + basename[:-4] + "_shuttle.mp4"
    csvout = out_dir + basename[:-4] + "_shuttle.csv"

    predict_shuttle(vid, 'model906_30', vidout, csvout)

Streaming output truncated to the last 5000 lines.
__________________________________________________________________________________________________
Beginning predicting......
Prediction time: 29.879156589508057 secs
Done......
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 9, 288, 512  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 64, 288, 512  5248        ['input_1[0][0]']                
                                )                                                                 
                                                             